In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from numpy.linalg import norm
import numpy as np
import os
from traitlets import default
import streamlit as st
#from streamlit_option_menu import option_menu
from sklearn.feature_extraction.text import TfidfVectorizer
from utils import cosine_similarity
#from page1 import page1_1,page1_2,page1_3,page1_4,page1_5
from st_aggrid import GridOptionsBuilder, AgGrid, GridUpdateMode, DataReturnMode
from gensim.parsing.preprocessing import STOPWORDS
from nltk.tokenize import word_tokenize
import regex as re
from rank_bm25 import *
import warnings
warnings.filterwarnings('ignore')
import nltk
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [37]:
URLs = ["https://medium.com/towards-data-science/statistics-for-people-in-a-hurry-a9613c0ed0b",
       "https://medium.com/towards-data-science/a-quick-guide-to-relational-algebra-operators-in-dbms-1ff2ddecaad7",
        "https://medium.com/grokking-the-tech-interview/the-top-data-structures-you-should-know-for-your-next-coding-interview-36af0831f5e3",
        "https://medium.com/cometheartbeat/the-5-computer-vision-techniques-that-will-change-how-you-see-the-world-1ee19334354b",
]

filename = """C:/Users/ai/OneDrive/Desktop/Temp/Ritvick/ir-app/data/IR Links.txt"""
URLs = []
with open(filename) as file:
    for line in file:
        URLs.append(line.rstrip())
URLs = URLs[1:]
soups = []
for url in URLs:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    soups.append(soup)

In [54]:
documents = []
titles = []

for soup in soups:
    sections = soup.find_all('section')
    story_paragraphs = []
    section_titles = []
    for section in sections:
        paragraphs = section.find_all('p')
        for paragraph in paragraphs:
            story_paragraphs.append(paragraph.text)

        subs = section.find_all('h1')
        for sub in subs:
            section_titles.append(sub.text)
    try:
        titles.append(section_titles[0])
    except:
        titles.append("No Title")
        documents.append("No Content")
        continue
    
    document = ""
    for para in story_paragraphs:
        document = document+ para + "\n"
    documents.append(document)


In [55]:
len(titles),len(URLs)

(349, 349)

In [56]:
sum([1 if doc=="" else 0 for doc in documents])

0

In [57]:
df = {}
df["url"] = URLs
df["title"] = titles
df["content"] = documents
df = pd.DataFrame(df)
df.to_csv('./data/data.csv')

## TF-IDF

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words={'english'})
indices = vectorizer.fit_transform(documents).toarray()

In [5]:
def cosine_similarity(query,keys):
    p1 = query.dot(keys)
    p2 = norm(keys,axis=0)*norm(query)
    return p1/p2

In [6]:
def recommend(text):
    index = vectorizer.transform([text]).toarray()[0]
    index = np.expand_dims(index,axis=-1)
    sim_scores = cosine_similarity(indices,index)
    print(sim_scores)
    

In [7]:
recommend("data")

[[0.05783263]
 [0.03906388]
 [0.12344652]
 [0.019242  ]]


## BM25

In [9]:
def remove_specials(documents):
    cleaned = list()
    for text in documents:
        word = re.sub("[0-9a-zA-Z]"," ",text)
        cleaned.append(word)
    return cleaned

def remove_stopwords(documents):
    cleaned = list()
    for text in documents:
        text_tokens = word_tokenize(text)
        tokens_without_sw = [word for word in text_tokens if not word in STOPWORDS]
        str_t = " ".join(tokens_without_sw)
        cleaned.append(str_t)
    return cleaned

def preprocess(documents):
    cleaned = remove_specials(documents)
    cleaned = remove_stopwords(documents)
    return cleaned

def process_documents():
    
    documents_cleaned = preprocess(documents)
    bm25_index = prepare_indices(documents_cleaned)
    return bm25_index

def prepare_indices(documents):
    
    tokenized_corpus = [doc.split(" ") for doc in documents]
    bm25_index = BM25Okapi(tokenized_corpus)
    return bm25_index

def recommend_bm(text,top_k):
    text = preprocess([text])[0]
    tokenized_query = text.split(" ")
    scores = bm25_index.get_scores(tokenized_query)

    recs = np.argsort(scores)[::-1][:top_k]
    return recs

bm25_index = process_documents()


In [10]:
recommend_bm("data",3)

array([2, 0, 1], dtype=int64)